# Parser final:

In [1]:
import requests
import json
import pandas as pd
import random

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

def parser(otkuda, kuda, date):
    
    converter = {"Алматы":"ALA",
    "Нур-Султан":"NQZ",
    "Караганда":"KGF",
    "Актау":"SCO",
    "Актобе":"AKX",
    "Атырау":"GUW",
    "Балхаш":"BXH",
    "Костанай":"KSN",
    "Кокшетау":"KOV",
    "Павлодар":"PWQ",
    "Петропавловск":"PPK",
    "Семей":"PLX",
    "Шымкент":"CIT",
    "Талдыкорган":"TDK",
    "Тараз":"DMB",
    "Уральск":"URA",
    "Усть-Каменогорск":"UKK",
    "Жезказган":"DZN"}
    
    origin = converter[otkuda]
    destination = converter[kuda]
    month = date
    
    url_initial = "http://api.travelpayouts.com/v2/prices/month-matrix?currency=kzt&origin={}&destination={}&show_to_affiliates=false&month={}&token=200a89e6d3c3eea2c02ea3b8e200e629"
    url_needed = url_initial.format(origin, destination, month)
    
    response = requests.request("GET", url_needed)
    data = json.loads(response.text)
    
    prices = []
    from_ = []
    to_ = []
    dates = []
    trip_class = []
    duration = []
    distance = []
    company = []

    for i in data["data"]:
        prices.append(i["value"])
        from_.append(i["origin"])
        to_.append(i["destination"])
        dates.append(i["depart_date"])
        trip_class.append(i["trip_class"])
        duration.append(i["duration"])
        distance.append(i["distance"])
        company.append(i["gate"])

    dictionary = {"Date": dates, "Price": prices, "From": from_, "To": to_, "Class": trip_class, "Duration": duration, "Distance": distance, "Company": company}
    df = pd.DataFrame(dictionary)
    
    random_number = random.randint(0, 100)
    file_dias = "data" + str(random_number) + ".xlsx"
    path = "C:\\Users\\dias_\\Downloads\\data{}.xlsx"
    path = path.format(str(random_number))
    
    df.to_excel(path, index=False)
    
    
    ###################################################################################################################
    #Email sending:
    ###################################################################################################################
    
    email_user = 'diaszhandar@gmail.com'
    email_password = '2312makpal'
    email_send = 'diaszhandar@gmail.com'
    subject = 'Парсинг авиалиний успешно сделан!'
    
    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_send
    msg['Subject'] = subject
    
    body = "Добрый вечер Диас, \nВаши данные успешно собраны, можете скачать в удобном excel формате.\nХорошего вам дня\n"
    msg.attach(MIMEText(body, "plain"))
    
    filename=file_dias
    attachment  =open(filename,'rb')
    part = MIMEBase('application','vnd.ms-excel')
    part.set_payload((attachment).read())
    attachment.close()
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment, filename= "+filename)
    msg.attach(part)
    
    text = msg.as_string()
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_user, email_password)
    server.sendmail(email_user, email_send, text)
    server.quit()
    
    
    

In [2]:
parser("Костанай", "Атырау", "2020-10-10")